In [2]:
install.packages("readr")
library(readr)

install.packages("dplyr")
library(dplyr)

Instalowanie pakietu w 'C:/Users/karol/AppData/Local/R/win-library/4.2'
(ponieważ 'lib' nie jest określony)



pakiet 'readr' został pomyślnie rozpakowany oraz sumy MD5 zostały sprawdzone


Warning message:
"nie można usunąć poprzedniej instalacji pakietu 'readr'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem z kopiowaniem C:\Users\karol\AppData\Local\R\win-library\4.2\00LOCK\readr\libs\x64\readr.dll do C:\Users\karol\AppData\Local\R\win-library\4.2\readr\libs\x64\readr.dll: Permission denied"
Warning message:
"przywrócono 'readr'"



Pobrane pakiety binarne są w
	C:\Users\karol\AppData\Local\Temp\RtmpoFGp0i\downloaded_packages


Warning message:
"pakiet 'readr' został zbudowany w wersji R 4.2.3"
Instalowanie pakietu w 'C:/Users/karol/AppData/Local/R/win-library/4.2'
(ponieważ 'lib' nie jest określony)



pakiet 'dplyr' został pomyślnie rozpakowany oraz sumy MD5 zostały sprawdzone


Warning message:
"nie można usunąć poprzedniej instalacji pakietu 'dplyr'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem z kopiowaniem C:\Users\karol\AppData\Local\R\win-library\4.2\00LOCK\dplyr\libs\x64\dplyr.dll do C:\Users\karol\AppData\Local\R\win-library\4.2\dplyr\libs\x64\dplyr.dll: Permission denied"
Warning message:
"przywrócono 'dplyr'"



Pobrane pakiety binarne są w
	C:\Users\karol\AppData\Local\Temp\RtmpoFGp0i\downloaded_packages


Warning message:
"pakiet 'dplyr' został zbudowany w wersji R 4.2.3"

Dołączanie pakietu: 'dplyr'


Następujące obiekty zostały zakryte z 'package:stats':

    filter, lag


Następujące obiekty zostały zakryte z 'package:base':

    intersect, setdiff, setequal, union




**Funkcje do konwersji danych**

In [3]:
convert_commas_to_dots <- function(df) {
  # Zamiana przecinków na kropki w kolumnach typu
  #character tylko jeśli zawierają przecinek
  df[] <- lapply(df, function(x) {
    if (is.character(x) && any(grepl(",", x))) {
      x <- gsub(",", ".", x)
      num_x <- suppressWarnings(as.numeric(x))
      if (all(is.na(num_x) & !is.na(x))) {
        return(x) # Jeśli wszystkie wartości stają się NA, pozostaw jako tekst
      } else {
        return(num_x) # Konwersja na numeric, jeśli możliwe
      }
    } else {
      return(x)
    }
  })
  return(df)
}

convert_numeric_to_integer <- function(df) {

  #Zamiana wartości na integer, jeśli nie zawierają
  #kropki i nie są wszystkimi NA 

  df[] <- lapply(df, function(x) {
    if (is.numeric(x) && all(!grepl("\\.", as.character(x)), na.rm = TRUE)) { # nolint
      return(as.integer(x))
    } else {
      return(x)
    }
  })
  return(df)
}

In [4]:
convert <- function(df) {
  df <- convert_commas_to_dots(df)
  df <- convert_numeric_to_integer(df)
  return(df)
}

**Zaczytanie danych**

In [5]:
teams <- read_csv2("teams.csv")
glimpse(teams)

ℹ Using "','" as decimal and "'.'" as grouping mark. Use `read_delim()` for more control.

Rows: 1324 Columns: 133
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
chr  (13): TM TAG, OPP TAG, TM NAME, OPP NAME, DATE, H/A, W/L, RIM PPS, MID ...
dbl (120): ROUND, MIN, PTS, 2PTM, 2PTA, 2PT%, 3PTM, 3PTA, 3PT%, FGM, FGA, FG...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 1,324
Columns: 133
$ ROUND              <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, …
$ `TM TAG`           <chr> "CZB", "ASV", "MTA", "PAR", "EFE", "FCB", "BER", "B…
$ `OPP TAG`          <chr> "ASV", "CZB", "PAR", "MTA", "FCB", "EFE", "BAY", "B…
$ `TM NAME`          <chr> "Crvena Zvezda Meridianbet Belgrade", "LDLC ASVEL V…
$ `OPP NAME`         <chr> "LDLC ASVEL Villeurbanne", "Crvena Zvezda Meridianb…
$ DATE               <chr> "05.10.2023", "05.10.2023", "05.10.2023", "05.10.20…
$ `H/A`              <chr> "H", "A", "H", "A", "A", "H", "A", "H", "H", "A", "…
$ `W/L`              <chr> "W", "L", "W", "L", "L", "W", "L", "W", "L", "W", "…
$ MIN                <dbl> 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 2…
$ PTS                <dbl> 94, 73, 96, 81, 74, 91, 68, 80, 79, 82, 82, 85, 88,…
$ `2PTM`             <dbl> 16, 17, 27, 26, 22, 32, 16, 23, 23, 18, 18, 18, 19,…
$ `2PTA`             <dbl> 37, 29, 43, 39, 40, 42, 30, 36, 39, 39, 36, 34, 35,…
$ `2PT%`       

In [6]:
players <- read_csv2("players.csv")
glimpse(players)

ℹ Using "','" as decimal and "'.'" as grouping mark. Use `read_delim()` for more control.

Rows: 7883 Columns: 85
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
chr (14): TM TAG, NAME, ROLE, NAT, HEIGHT, TM NAME, OPP NAME, DATE, H/A, W/L...
dbl (71): AGE, ROUND, MIN, PTS, 2PTM, 2PTA, 3PTM, 3PTA, FGM, FGA, FTM, FTA, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 7,883
Columns: 85
$ `TM TAG`       <chr> "CZB", "CZB", "CZB", "CZB", "CZB", "CZB", "CZB", "CZB",…
$ NAME           <chr> "Adam Hanga", "Branko Lazic", "Dalibor Ilic", "Luka Mit…
$ ROLE           <chr> "F", "SG", "F", "PF", "PF", "C", "PG", "SG", "C", "F/PF…
$ NAT            <chr> "HUN", "SRB", "BIH", "SRB", "SRB", "USA", "SRB", "SRB",…
$ HEIGHT         <chr> "201", "195", "206", "206", "203", "213", "195", "191",…
$ AGE            <dbl> 35, 35, 24, 31, 38, 30, 37, 33, 34, 32, 33, 28, 30, 35,…
$ ROUND          <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ `TM NAME`      <chr> "Crvena Zvezda Meridianbet Belgrade", "Crvena Zvezda Me…
$ `OPP NAME`     <chr> "LDLC ASVEL Villeurbanne", "LDLC ASVEL Villeurbanne", "…
$ DATE           <chr> "05.10.2023", "05.10.2023", "05.10.2023", "05.10.2023",…
$ `H/A`          <chr> "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", …
$ `W/L`          <chr> "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", …
$ MIN           

**Konwersja danych**

In [7]:
teams <- convert(teams)
glimpse(teams)

Rows: 1,324
Columns: 133
$ ROUND              <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, …
$ `TM TAG`           <chr> "CZB", "ASV", "MTA", "PAR", "EFE", "FCB", "BER", "B…
$ `OPP TAG`          <chr> "ASV", "CZB", "PAR", "MTA", "FCB", "EFE", "BAY", "B…
$ `TM NAME`          <chr> "Crvena Zvezda Meridianbet Belgrade", "LDLC ASVEL V…
$ `OPP NAME`         <chr> "LDLC ASVEL Villeurbanne", "Crvena Zvezda Meridianb…
$ DATE               <chr> "05.10.2023", "05.10.2023", "05.10.2023", "05.10.20…
$ `H/A`              <chr> "H", "A", "H", "A", "A", "H", "A", "H", "H", "A", "…
$ `W/L`              <chr> "W", "L", "W", "L", "L", "W", "L", "W", "L", "W", "…
$ MIN                <int> 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 2…
$ PTS                <int> 94, 73, 96, 81, 74, 91, 68, 80, 79, 82, 82, 85, 88,…
$ `2PTM`             <int> 16, 17, 27, 26, 22, 32, 16, 23, 23, 18, 18, 18, 19,…
$ `2PTA`             <int> 37, 29, 43, 39, 40, 42, 30, 36, 39, 39, 36, 34, 35,…
$ `2PT%`       

In [8]:
players <- convert(players)
glimpse(players)

Rows: 7,883
Columns: 85
$ `TM TAG`       <chr> "CZB", "CZB", "CZB", "CZB", "CZB", "CZB", "CZB", "CZB",…
$ NAME           <chr> "Adam Hanga", "Branko Lazic", "Dalibor Ilic", "Luka Mit…
$ ROLE           <chr> "F", "SG", "F", "PF", "PF", "C", "PG", "SG", "C", "F/PF…
$ NAT            <chr> "HUN", "SRB", "BIH", "SRB", "SRB", "USA", "SRB", "SRB",…
$ HEIGHT         <chr> "201", "195", "206", "206", "203", "213", "195", "191",…
$ AGE            <int> 35, 35, 24, 31, 38, 30, 37, 33, 34, 32, 33, 28, 30, 35,…
$ ROUND          <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ `TM NAME`      <chr> "Crvena Zvezda Meridianbet Belgrade", "Crvena Zvezda Me…
$ `OPP NAME`     <chr> "LDLC ASVEL Villeurbanne", "LDLC ASVEL Villeurbanne", "…
$ DATE           <chr> "05.10.2023", "05.10.2023", "05.10.2023", "05.10.2023",…
$ `H/A`          <chr> "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", "H", …
$ `W/L`          <chr> "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", …
$ MIN           

In [11]:
# Sumowanie wszystkich statystyk po nazwie zespołu
teams_total <- teams %>%
  group_by(`TM NAME`) %>%
  summarise(across(where(is.numeric), sum, na.rm = TRUE))

glimpse(teams_total)

Rows: 18
Columns: 127
$ `TM NAME`          <chr> "ALBA Berlin", "AS Monaco", "Anadolu Efes Istanbul"…
$ ROUND              <int> 1190, 1580, 1260, 1560, 1190, 1190, 1580, 1190, 175…
$ MIN                <int> 13600, 15770, 14250, 15650, 13600, 13700, 15650, 13…
$ PTS                <int> 5182, 6382, 5984, 6600, 5528, 5318, 6326, 5348, 689…
$ `2PTM`             <int> 1282, 1700, 1494, 1504, 1360, 1230, 1712, 1298, 151…
$ `2PTA`             <int> 2522, 3204, 2692, 2702, 2478, 2232, 3136, 2430, 273…
$ `2PT%`             <dbl> 34.70, 41.70, 39.04, 43.46, 37.70, 37.90, 42.92, 36…
$ `3PTM`             <int> 604, 610, 696, 848, 628, 682, 656, 672, 940, 488, 7…
$ `3PTA`             <int> 1720, 1746, 1880, 2286, 1840, 1860, 1824, 1876, 239…
$ `3PT%`             <dbl> 24.12, 27.04, 25.94, 28.40, 22.98, 25.06, 28.16, 24…
$ FGM                <int> 1886, 2310, 2190, 2352, 1988, 1912, 2368, 1970, 245…
$ FGA                <int> 4242, 4950, 4572, 4988, 4318, 4092, 4960, 4306, 512…
$ `FG%`           

In [ ]:
#Usunąć kolumny, które nie mają sensu po zsumowaniu np. ROUND